### <center>Python'da Backtrader ile geriye dönük test yapma</center>
#### <center>Serkan Polat</center>

## Strateji optimizasyonu ve geriye dönük test

Optimizasyon, bir strateji uzmanının, stratejiyi yönlendiren parametrelere ve formüllere ince ayar yaparak stratejisinin sonuçlarını iyileştirmesine olanak tanır. Optimizasyon, istenen optimizasyon hedefine ulaşmak için tek bir parametreye veya bir dizi parametreye ince ayar yapılarak yapılabilir. Strateji optimizasyon hedefine bir örnek, üretilen toplam karı maksimuma çıkarmak olabilir. Optimizasyonun bir diğer amacı dezavantajları en aza indirmek olabilir.

Optimizasyon amacından bahsetmişken, optimizasyonun iki ucu keskin bir kılıç gibi olduğunun da bilincinde olmak gerekir. İstenilen eşitlik eğrisini elde etmek için optimizasyon sırasında eğitim verilerine birçok kural uygulanırsa, bu durum verilerin aşırı uyumuyla sonuçlanır ve modelin test ve gelecek veriler üzerindeki tahmin yeteneğini kaybetmesi muhtemeldir.

Basit bir ifadeyle, bir ticaret stratejisinin geriye doğru test edilmesi, bir ticaret hipotezinin/stratejisinin önceki zaman dilimlerinde test edilmesi sürecidir. Bir tüccar, yıllar sürebilecek ileriki zaman dilimi için (performansı değerlendirmek için) bir strateji uygulamak yerine, ticaret stratejisini ilgili geçmiş veriler üzerinde simüle edebilir.

- ### Geriye Dönük Testçi Türleri

Bu hipotezi test etmek için kullanılan genel olarak iki tür geriye dönük test sistemi vardır; araştırma geri test uzmanları ve olay odaklı geri test uzmanları. Şimdi bunları ayrıntılı olarak inceleyelim.

1. #### Araştırma Geri Testçileri

Bu araçlar, pazar etkileşiminin tüm yönlerini tam olarak simüle etmez ancak potansiyel strateji performansının hızlı bir şekilde belirlenmesini sağlamak için tahminler yapar. Bu araçlar sıklıkla geriye dönük test ve uygulama için kullanılsa da, gün içi ticarete daha yüksek frekanslarda yaklaşan stratejiler için uygun değildir. Daha gerçekçi bir ortamda sıkı bir arka teste girmeden önce tüm strateji fikirleri için "ilk taslağı" almak amacıyla profesyonel kantitatif ticaret endüstrisinde yaygın olarak kullanılırlar.

2. #### Olay Odaklı Geriye Dönük Test

Olaya dayalı geriye dönük testte, otomatik ticaret stratejisi, gerçek zamanlı bir piyasa beslemesine ve bir komisyoncuya bağlanır; böylece sistem, yeni bir ticaret sinyali oluşturmak için bir olayı tetikleyen bir sisteme gönderilecek olan yeni piyasa bilgilerini alır. Bu sistemler sürekli bir döngü halinde çalışır ve tarihi veri işleyici, aracılık simülatörü gibi alt bileşenlere sahip olabilir; canlı yürütmeye çok benzer şekilde geriye dönük test yapılmasına izin verir. Tek dezavantajı bu sistemlerin karmaşık bir tasarıma sahip olması ve hata yapmaya daha yatkın olmasıdır.

# Python'da Backtrader ile geriye dönük test yapma

Geriye dönük test, etkili ticaret sistemi geliştirmenin önemli bir bileşenidir. Belirli bir strateji tarafından tanımlanan kurallar kullanılarak geçmişte gerçekleşmiş olabilecek ticaretlerin tarihsel verilerle yeniden yapılandırılmasıyla gerçekleştirilir. Backtrader kütüphanesi Python'da bu tür testleri yürütmek için araçlar sağlar.

Genel yaklaşımımız şu şekildedir:

1. Veri Toplama.
2. Geriye Dönük Test Stratejimizi Tanımlamak.
3. Karşılaştırmayla Karşılaştırmak ve Karşılaştırmak için Performans İstatistiklerinin Çıkarılması.


## Örnek 1: Hareketli Ortalama Geçiş Stratejisi

In [1]:
! pip install backtrader >Null


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import backtrader as bt
import yfinance as yf
import datetime

# Adım 2: Bir Ticaret Stratejisi Oluşturun
class MovingAverageCrossStrategy(bt.Strategy):
    params = (
        ("short_window", 40),
        ("long_window", 100),
    )

    def __init__(self):
        self.data_close = self.datas[0].close
        self.short_ma = bt.indicators.SimpleMovingAverage(
            self.data_close, period=self.params.short_window
        )
        self.long_ma = bt.indicators.SimpleMovingAverage(
            self.data_close, period=self.params.long_window
        )
        self.crossover = bt.indicators.CrossOver(self.short_ma, self.long_ma)

    def next(self):
        if self.crossover > 0:  # kısa MA uzun MA'yı geçerse
            self.buy()
        elif self.crossover < 0:  # kısa MA uzun MA'nın altına geçerse
            self.sell()

# 3. Adım: yfinance'ı kullanarak Geçmiş Verileri Yükleyin
data_df = yf.download('THYAO.IS', start='2020-01-01', end='2022-01-01')
data = bt.feeds.PandasData(dataname=data_df)

# Adım 4: Backtest'i Ayarlayın
cerebro = bt.Cerebro()
cerebro.addstrategy(MovingAverageCrossStrategy)
cerebro.adddata(data)
cerebro.broker.set_cash(100000)  # Örneğin. 100.000 nakitle başla

# 5. Adım: Ek Ayarlar Ekleyin (İsteğe Bağlı)
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)
cerebro.broker.setcommission(commission=0.001)  # %0,1 komisyon (örnek)

# Adım 6: Geri Testi Çalıştırın
print('Başlangıç ​​Portföy Değeri: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Bitiş Portföy Değeri: %.2f' % cerebro.broker.getvalue())

# Adım 7: Sonuçları Görselleştirin
cerebro.plot()


[*********************100%***********************]  1 of 1 completed
Başlangıç ​​Portföy Değeri: 100000.00
Bitiş Portföy Değeri: 102770.70


<IPython.core.display.Javascript object>

[[<Figure size 640x480 with 5 Axes>]]

Backtrader tarafından oluşturulan grafik, ticaret stratejisinin geriye dönük test dönemi boyunca performansının görsel bir temsilini sağlar. Olay örgüsünün ana bileşenlerini şu şekilde yorumlayabilirsiniz:

### 1. **Fiyat Tablosu**
- **Mum/OHLC Çubukları**: Varlığın her dönem için açılış, en yüksek, en düşük ve kapanış fiyatlarını temsil eder.
- **Hareketli Ortalamalar**: Stratejinizde kullanılırsa, hareketli ortalamalar fiyat grafiğinin üzerine çizilecektir.
- **Alış/Satış İşaretleri**: Oklar veya işaretler, stratejinin alım veya satım emirlerini nerede yürüttüğünü gösterir.

### 2. **Portföy Değeri**
- **Nakit**: Yatay çizgi, aracılık hesabında bulunan nakdi temsil eder.
- **Portföy Değeri**: Çizgi grafik, nakit ve açık pozisyonların zaman içindeki değeri de dahil olmak üzere toplam portföy değerini gösterir.

### 3. **İşlemler**
- **Kapalı İşlemler**: Noktalar veya işaretler, çizim ayarlarınıza bağlı olarak işlemlerin nerede kapatıldığını gösterebilir.

### 4. **Göstergeler**
- Ek göstergeler (MACD, RSI vb. gibi) kullandıysanız, bunlar zaman içindeki değerlerini gösterecek şekilde ayrı panellerde gösterilebilir.

### 5. **Yakınlaştırma ve Kaydırma**
- Belirli zaman dilimlerini ayrıntılı olarak incelemek için yakınlaştırabilir/uzaklaştırabilir ve çizimi kaydırabilirsiniz.

### Nasıl Analiz Edilir:
- **Karlılık**: Stratejinin zaman içindeki karlılığını değerlendirmek için Portföy Değeri çizgisini gözlemleyin.
- **Düşüş**: Portföy Değerinin düştüğü, düşüşleri gösteren dönemleri arayın.
- **Ticaret Gerçekleştirme**: İşlemlerin (alış/satış göstergelerinin) strateji mantığına göre amaçlanan pozisyonlarda yürütülmesini sağlayın.
- **Gösterge Davranışı**: Teknik göstergeler kullanılıyorsa davranışlarını gözlemleyin ve beklendiği gibi hesaplandığından emin olun.
- **Aşırı uyum**: Aşırı uyum konusunda dikkatli olun. Eğer strateji geriye dönük testlerde son derece iyi performans gösteriyorsa, geçmiş verilere göre fazla uygun olabilir ve canlı ticarette iyi performans göstermeyebilir.

### Ek İpuçları:
- **Risk Yönetimi**: Stratejinin, zararı durdurma seviyelerini belirlemek ve tek bir işlemde portföyün yalnızca küçük bir yüzdesini riske atmak gibi sağlam risk yönetimi ilkelerine bağlı olduğundan emin olun.
- **Hassasiyet**: Stratejinin parametrelerdeki değişikliklere (hareketli ortalama dönemleri gibi) ne kadar duyarlı olduğunu test edin ve stratejinin sağlam olduğundan ve test edilen döneme göre aşırı optimize edilmediğinden emin olun.
- **Örnek Dışı Test**: Görünmeyen veriler üzerindeki performansını doğrulamak için stratejiyi örnek dışı veriler üzerinde doğrulayın.

Backtrader, özel mantık tanımlayarak kendi ticaret stratejilerinizi oluşturmanıza olanak tanıyan esnek bir Python kütüphanesidir. Kendi başına önceden tanımlanmış ticaret stratejileriyle birlikte gelmez, ancak çok çeşitli stratejileri tanımlamanıza olanak tanıyan sağlam bir çerçeve ve yerleşik göstergeler sağlar.

Backtrader'ı kullanarak uygulayabileceğiniz bazı strateji türleri şunlardır:


| Strateji                          | Kısa Açıklama |
|:---------------------------------|:-------------------|
| Hareketli Ortalama Kesişimi      | Kısa vadeli hareketli ortalama uzun vadeli hareketli ortalama kesiştiğinde al/sat yapma stratejisi. |
| Ortalama Tersiyon                | Bir varlık fiyatının ortalama veya diğer ilgili bir referans noktasından sapmasına dayalı olarak işlem yapma. |
| Momentum                         | Bir varlığın fiyatının hızlanmasına dayalı olarak al/sat yapma. |
| Bollinger Bantları               | Bir hareketli ortalamanın üzerine ve altına yerleştirilen volatilite bantlarını kullanarak aşırı alım/aşırı satım koşullarını tanımlama. |
| MACD                             | İki hareketli ortalama arasındaki yakınsama/ayrışmayı kullanarak al/sat sinyalleri üretme. |
| RSI                              | Göreceli Güç Endeksi'ni kullanarak aşırı alım/aşırı satım koşullarını tanımlama. |
| Çift Ticaret                     | İki ilişkili varlığın fiyat oranı üzerinde işlem yapma. |
| Arbitraj                        | İlgili varlıklar veya piyasalar arasındaki fiyat farklılıklarından yararlanma. |
| Makine Öğrenme Tabanlı           | Tahmin modellerini kullanarak ticaret sinyalleri üretme. |
| Opsiyon Stratejileri            | Opsiyonları kullanarak straddle, strangle, spread gibi stratejileri uygulama. |
| Dolar Maliyet Ortalama           | Fiyattan bağımsız olarak belirli aralıklarla sabit bir miktar yatırım yapma. |
| Portföy Dengesi                 | İstenen varlık tahsisini korumak için bir portföyü yeniden dengeleme. |
| Mevsimsellik                    | Piyasada mevsimsel desenlere veya döngülere dayalı olarak işlem yapma. |
| İstatistik Arbitraj             | İstatistik yöntemleri kullanarak getiri üretme, örneğin, eşbütünleşme gibi. |
| Trend Takip                    | Hakim olan trendin yönünü tanımlama ve ticaret yapma. |
| Volatilite Patlaması            | Ani volatilite artışlarına dayalı olarak işlem yapma. |
| Stop Loss / Kar Al              | Ticaretleri yönetmek için stop-loss veya kar alma seviyelerini kullanan stratejileri uygulama. |
| Izgara Ticareti                 | Fiyat yönünü tahmin etmeksizin önceden belirlenmiş aralıklarda (ızgara seviyelerinde) al/sat yapma. |
| Sapma Ticareti                  | Fiyat ile bir osilatör arasındaki sapmayı tanımlama ve buna göre işlem yapma. |
| Hedge                           | Ticarette olası kayıpları azaltmayı amaçlayan stratejileri uygulama. |
| Çoklu Zaman Çerçevesi Stratejileri | Ticaret kararları vermek için birden fazla zaman çerçevesini kullanma. |
| Çoklu Varlık Stratejileri       | Birden fazla, muhtemelen ilişkili, varlık içeren ticaret stratejileri uygulama. |
| Piyasa Yapıcılığı                | Alım-satım fiyatlarını sürekli olarak teklif ederek teklif-fiyat arasındaki farktan kar elde etme. |
| Yüksek Frekanslı Ticaret         | Genellikle yüksek kaldıraçla küçük fiyat farklarından faydalanmayı amaçlayan stratejileri uygulama. |
| Emir Akışı                      | Emir akış bilgisi analizi temelinde ticaret yapma. |
| Duygu Analizi                  | Ticaret kararları vermek için duygu analizi kullanma. |
| Desen Tanıma                   | Tanınmış desenlere dayalı olarak ticaret yapma (örneğin, baş ve omuzlar, üçgenler vb.). |
| Wyckoff Yöntemi              | Birikim ve dağıtımın Wyckoff yöntemi prensiplerine dayalı olarak ticaret yapma. |
| Elliott Dalga Teorisi         | Piyasa yapılarını tanımlama temelinde ticaret yapma, bu yapılar "dalgalar" olarak adlandırılır. |
| Fibonacci Geri Çekilmeleri   | Potansiyel ters dönüş seviyelerini tanımlamak için Fibonacci seviyelerini kullanma. |
| Pivot Noktaları               | Pivot noktaları ve destek/direnç seviyelerine dayalı olarak ticaret yapma. |
| Ichimoku Bulutu               | Trendleri ve bunların güçlerini tanımlamak için Ichimoku bulutunu kullanma. |
| Heikin Ashi                  | Heikin Ashi mumlarını kullanarak trendleri ve ters dönüşleri tanımlama. |
| Gann Teorisi                | W.D. Gann'ın teorilerine dayalı stratejiler uygulama, Gann açılarının kullanımı gibi. |
| Piyasa Profili              | Piyasa profilini kullanarak fiyat gelişmelerini ve piyasadaki değer alanlarını anlama. |
| Hacim Profili               | Farklı fiyat seviyelerindeki hacmin dağılımına dayalı olarak ticaret yapma. |
| VWAP (Hacme Ağırlıklı Ortalama Fiyat) | VWAP'a yakın ticaret yapmayı amaçlayan stratejiler uygulama. |
| TWAP (Zaman Ağırlıklı Ortalama Fiyat) | TWAP'a yakın ticaret yapmayı amaçlayan stratejiler uygulama. |
| Alpha ve Beta               | Alfa (aktif getiri) ve beta (piyasa getirisine duyarlılık) temelinde stratejiler geliştirme. |
| Risk Paritesi               | Risk temelinde sermaye tahsisi yapma, genellikle volatiliteyi bir ölçüt olarak kullanma. |
| Akıllı Beta                  | Alternatif endeks oluşturma kurallarını kullanarak stratejiler uygulama. |
| Carry Ticareti              | Farklı menkul kıymetler arasındaki getiri farkından kar elde etmeyi amaçlayan stratejiler uygulama. |
| Gamma Kazanç                | Bir opsiyonun delta değişikliklerinden kar elde etmeyi amaçlayan stratejiler uygulama. |
| Delta Hedge                 | Bir pozisyonun fiyat riskini dengelemeyi amaçlayan stratejiler uygulama. |
| Takvim Yayılımlar            | Farklı son kullanma tarihlerine sahip iki opsiyonun uzun ve kısa pozisyonunu oluşturan stratejiler uygulama. |
| Kelebek Yayılımlar            | Farklı kullanım fiyatlarına sahip birden fazla opsiyon işlemi içeren stratejiler uygulama. |
| Demir Turna Yayılımları         | Aynı son kullanma tarihine sahip dört opsiyon işlemi içeren stratejiler uygulama. |
| Yönlendirme Stratejileri        | Opsiyon pozisyonlarını ayarlamayı içeren stratejiler uygulama. |
| Kapalı Çağrı                   | Sahip olunan bir hisse senedi üzerine alım opsiyonları yazmayı içeren stratejiler uygulama. |
| Koruyucu Alım                  | Sahip olunan hisse senedine karşı düşüşe karşı sigorta olarak put opsiyonları satın almayı içeren stratejiler uygulama. |
| Boğa Alım Yayılımı             | Aynı son kullanma tarihine sahip, farklı kullanım fiyatlarına sahip alım opsiyonlarını satın almayı ve satmayı içeren stratejiler uygulama. |
| Ayı Put Yayılımı               | Aynı son kullanma tarihine sahip, farklı kullanım fiyatlarına sahip put opsiyonlarını satın almayı ve satmayı içeren stratejiler uygulama. |
| Straddle                       | Aynı kullanım fiyatına ve son kullanma tarihine sahip hem bir alım hem de bir put opsiyonunu satın almayı veya satmayı içeren stratejiler uygulama. |
| Strangle                       | Aynı son kullanma tarihine sahip farklı kullanım fiyatlarına sahip hem bir alım hem de bir put opsiyonunu satın almayı veya satmayı içeren stratejiler uygulama. |
| Yaka                          | Altında yatırım yapılırken aynı anda bir koruyucu put opsiyonu almayı ve bir karşı tarafça örtülü çağrı yazmayı içeren stratejiler uygulama. |
| Turna                           | Aynı son kullanma tarihine sahip dört opsiyon sözleşmesini içeren stratejiler uygulama. |
| Kutu Yayılımı                   | Boğa çağrı yayılımı ve ayı put yayılımının bir kombinasyonunu içeren stratejiler uygulama. |
| Oran Yayılımı                   | Belirli bir oranda opsiyon almayı ve satmayı içeren stratejiler uygulama. |



## Örnek 2: Bollinger Bantları ve Göreceli Güç Endeksi (RSI)

Backtrader ile Bollinger Bantlarını ve Göreceli Güç Endeksi'ni (RSI) kullanarak daha karmaşık bir strateji oluşturalım. Bu strateji şunları sağlayacaktır:
- Fiyat alt Bollinger Bandının altında kapandığında ve RSI 30'un altında olduğunda (aşırı satış durumunu gösterir) satın alın.
- Fiyat üst Bollinger Bandının üzerinde kapandığında ve RSI 70'in üzerinde olduğunda satış yapın (aşırı alım durumunu gösterir).


### Not:
- Stratejinin sağlamlığından emin olmak için farklı verilerle ve çeşitli senaryolar altında doğrulandığından emin olun.
- Geçmiş performansın gelecekteki sonuçların göstergesi olmadığını ve geriye dönük testin bir ticaret stratejisi geliştirirken kullanılan birçok araçtan biri olması gerektiğini unutmayın.
- Ticaret stratejileri geliştirirken daima sağlam risk yönetimi ilkelerine bağlı kalın.

In [3]:
import backtrader as bt
import yfinance as yf
import datetime

# Strateji Oluşturun
class BollingerRSIStrategy(bt.Strategy):
    params = (
        ("bband_period", 20),
        ("rsi_period", 14),
        ("overbought", 70),
        ("oversold", 30),
        ("stddev", 2),
    )

    def __init__(self):
        self.bband = bt.indicators.BollingerBands(
            self.data.close, period=self.params.bband_period, devfactor=self.params.stddev
        )
        self.rsi = bt.indicators.RelativeStrengthIndex(
            period=self.params.rsi_period
        )

    def next(self):
        if self.data.close < self.bband.lines.bot and self.rsi < self.params.oversold:
            self.buy()
        elif self.data.close > self.bband.lines.top and self.rsi > self.params.overbought:
            self.sell()

# Geçmiş verileri indir
data_df = yf.download('THYAO.IS', start='2020-01-01', end='2022-01-01')
data = bt.feeds.PandasData(dataname=data_df)

# Bir Cerebro varlığı oluşturun
cerebro = bt.Cerebro(stdstats=False)

# Strateji ekle
cerebro.addstrategy(BollingerRSIStrategy)

# Veri Akışını Cerebro'ya Ekle
cerebro.adddata(data)

# İstediğiniz nakit başlangıcını ayarlayın
cerebro.broker.set_cash(100000.0)

# Bahis miktarına göre bir Sabit Boyut boyutlandırıcı ekleyin
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)

# Komisyonu ayarlayın
cerebro.broker.setcommission(commission=0.001)

# Başlangıç ​​koşullarını yazdırın
print('Başlangıç ​​Portföy Değeri: %.2f' % cerebro.broker.getvalue())

# Her şeyin üzerinden geç
cerebro.run()

# Nihai sonucu yazdırın
print('Bitiş Portföy Değeri: %.2f' % cerebro.broker.getvalue())

# Sonucu çizin
cerebro.plot(iplot=True)

[*********************100%***********************]  1 of 1 completed
Başlangıç ​​Portföy Değeri: 100000.00
Bitiş Portföy Değeri: 159804.48


<IPython.core.display.Javascript object>

[[<Figure size 640x480 with 3 Axes>]]

### Ticarette Performans ve Risk Metriklerini Anlamaya İlişkin Kısa Not

Ticarette bir stratejinin etkinliğini ve riskini değerlendirmek çok önemlidir. Performans ölçümleri bir stratejinin karlılığını ve tutarlılığını değerlendirmeye yardımcı olurken, risk ölçümleri de stratejinin potansiyel güvenlik açıklarına ilişkin içgörüler sağlar. Her iki kategorideki bazı temel metrikleri inceleyelim.

#### Bölüm 1: Performans Metrikleri

1. **İade Önlemleri**
   - **Toplam K/Z**: Bir stratejinin genel kar veya zararını değerlendirir.
   - **Ortalama K/Z**: Birim zaman başına ortalama kar veya zararı gösterir.
   - **Başarı Oranı (İsabet Oranı)**: Kârlı işlemlerin kaybedilen işlemlere oranını temsil eder.

2. **Riske Göre Ayarlanmış Önlemler**
   - **Sharpe Oranı**: Risk birimi başına aşırı getiriyi değerlendirir, ancak gelecekteki modellerin geçmişi taklit edeceğini varsaymak gibi sınırlamaları vardır.
   - **Sortino Oranı**: Negatif getirilerin standart sapmasına odaklanan Sharpe Oranının bir varyasyonu.
   - **Calmar Oranı**: Aynı dönemdeki maksimum düşüşe karşı üç yıllık ortalama yıllık getiri oranını dikkate alır.

3. **Göreceli Getiri Ölçüleri**
   - **Yukarı/Aşağı Yakalama Oranı**: Fonun yukarı/aşağı hareketleri sırasındaki getirisini referans getirisiyle karşılaştırır.
   - **Yukarı/Aşağı Yüzde Oranı**: Bir fonun olumlu/negatif dönemlerinde karşılaştırma ölçütünün ne sıklıkla üzerinde performans gösterdiğini ölçer.

#### Bölüm 2: Risk Ölçümleri

1. **Temel Risk Ölçümleri**
   - **Varyans**: Varlık volatilitesini yansıtacak şekilde getirinin beklenenden ne kadar saptığını gösterir.
   - **Maksimum Düşüş**: Yüzde olarak ifade edilen, zirveden dibe kadar en büyük kaybı temsil eder.

2. **Göreceli Risk Önlemleri**
   - **Korelasyon Katsayısı**: İki değişken arasındaki doğrusal ilişkinin gücünü ve yönünü gösterir.
   - **Beta**: Bir portföyün piyasaya göre oynaklığını ölçer.

3. **Kuyruk Riski Önlemleri**
   - **Riske Maruz Değer (VaR)**: Belirli bir dönem ve güven aralığı boyunca potansiyel kaybı tahmin eder.
   - **Koşullu Riske Maruz Değer (CVaR)**: VaR ile VaR'ı aşan kayıplar arasındaki ortalamalar, daha kapsamlı bir risk değerlendirmesi sağlar.

Bu ölçümleri anlamak ve uygulamak, yatırımcılara stratejilerini değerlendirme ve iyileştirme, getiri arayışı ile ihtiyatlı risk yönetimi arasında denge kurma gücü verecektir. Ticaret tarzınıza ve hedeflerinize uygun olduklarından emin olmak için daima her metriğin sınırlamalarını ve uygulanabilirliğini göz önünde bulundurun.

Python, ticaret stratejilerine yönelik performans ve risk ölçümlerini hesaplamak için çeşitli kütüphaneler sunar. Aşağıda dikkate değer olanlardan bazıları verilmiştir:


### 1. 'pyfolio'

'pyfolio', Quantopian Inc. tarafından geliştirilen, finansal portföylerin performans ve risk analizine yönelik bir Python kütüphanesidir. Performans ve riskin çeşitli yönlerini görselleştirmeye yönelik işlevsellik sağlar ve aynı zamanda Sharpe oranı gibi birçok ortak performans ölçümünü hesaplamak için yerleşik işlevlere sahiptir. Sortino oranı ve çok daha fazlası.


### 2. `ffn` - Python için Finansal İşlevler

'ffn', niceliksel finans alanında çalışanlar için birçok yararlı işlevi içeren bir kütüphanedir. Ayrıca yalnızca birkaç satır kodla Calmar oranı, maksimum düşüş ve çok daha fazlası gibi performans ölçümlerini hesaplamanıza da olanak tanır.


### 3. 'ampirik'

'ampirik', Quantopian tarafından geliştirilen ve çeşitli finansal risk ve performans ölçümleri sağlayan başka bir kütüphanedir. Kullanımı oldukça kolaydır ve Sharpe oranı, Sortino oranı vb. gibi ölçümleri hesaplayabilir.


### 4. `bt` - Python için Geriye Dönük Test

'bt', niceliksel ticaret stratejilerini test etmek için kullanılan Python için esnek bir geriye dönük test kütüphanesidir. Ayrıca çeşitli risk ve performans ölçümlerini hesaplamanıza da olanak tanır.


### 5. 'Quantlib'

'Quantlib' daha çok türevlere ve niceliksel finans hesaplamalarına odaklanır ancak çeşitli finansal hesaplamalara uyarlanabilir ve finansta karmaşık niceliksel yöntemler için yaygın olarak kullanılır.